In [30]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, InputLayer
import matplotlib.pyplot as plt 

In [44]:
df = pd.read_csv('../data/Data_A.csv', header=None)

columns=['midprice_up','ss_lob_1_p','ss_lob_1_v','bs_lob_1_p','bs_lob_1_v','ss_lob_2_p','ss_lob_2_v',
'bs_lob_2_p','bs_lob_2_v','ss_lob_3_p','ss_lob_3_v','bs_lob_3_p','bs_lob_3_v','ss_lob_4_p','ss_lob_4_v',
'bs_lob_4_p','bs_lob_4_v','c1','c2','c3','c4','c5']

df.columns = columns

In [4]:
price_cols = ['ss_lob_1_p','bs_lob_1_p','ss_lob_2_p','bs_lob_2_p','ss_lob_3_p','bs_lob_3_p','ss_lob_4_p','bs_lob_4_p']
vol_cols = ['ss_lob_1_v','bs_lob_1_v','ss_lob_2_v','bs_lob_2_v','ss_lob_3_v','bs_lob_3_v','ss_lob_4_v','bs_lob_4_v']
prev_mid_m_cols = ['c1', 'c2', 'c3', 'c4', 'c5']

price_df = df[price_cols]

min_price = price_df.min().min()
max_price = price_df.max().max()

vol_df = df[vol_cols]

min_vol = vol_df.min().min()
max_vol = vol_df.max().max()

price_transformer = lambda x: (x-min_price)/(max_price-min_price)
vol_transformer = lambda x: (x-min_vol)/(max_vol-min_vol)



In [5]:
df[vol_cols] = vol_transformer(df[vol_cols])
df[price_cols] = price_transformer(df[price_cols])


In [20]:
# flag for if there is buyside pressure, ie vol below>vol above
df['bs_pressure1'] = (df['bs_lob_1_v']>df['ss_lob_1_v']).astype(int)
df['bs_pressure2'] = ((df['bs_lob_1_v']+df['bs_lob_2_v'])>(df['ss_lob_1_v']+df['ss_lob_2_v'])).astype(int)

df['l1_diff']= df['ss_lob_1_v']-df['bs_lob_1_v']

df['avg_5'] = df[prev_mid_m_cols].mean(axis=1)
df['momentum_up'] = (df['avg_5']>0.5).astype(int)


df['vwap_bs_2'] = ((df['bs_lob_1_v']*df['bs_lob_1_p']) + (df['bs_lob_2_v']*df['bs_lob_2_p'])) /(df['bs_lob_1_v']+df['bs_lob_2_v'])
df['vwap_ss_2'] = ((df['ss_lob_1_v']*df['ss_lob_1_p']) + (df['ss_lob_2_v']*df['ss_lob_2_p'])) /(df['ss_lob_1_v']+df['ss_lob_2_v'])



##### Train with only L1 order book diff feature

- Only need 1 layer of NN for this with 1 node

In [ ]:
X =df[[
        'l1_diff'
        ]] 

Y = df['midprice_up']

model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(len(X.columns),)),
    keras.layers.Dense(1, activation="elu"),
    keras.layers.Dense(1, activation="sigmoid")
]    
)
# model.summary()
eta = 0.05
model.compile(optimizer=keras.optimizers.Adam(learning_rate=eta), loss="binary_crossentropy", metrics=["accuracy"])
model.fit(X, Y, batch_size=500, epochs=20)

Epoch 1/20
200/200 [==============================] - 1s 764us/step - loss: 0.6763 - accuracy: 0.5914
Epoch 2/20
200/200 [==============================] - 0s 749us/step - loss: 0.6273 - accuracy: 0.7186
Epoch 3/20
200/200 [==============================] - 0s 745us/step - loss: 0.6201 - accuracy: 0.7240
Epoch 4/20
200/200 [==============================] - 0s 760us/step - loss: 0.6193 - accuracy: 0.7289
Epoch 5/20
200/200 [==============================] - 0s 744us/step - loss: 0.6195 - accuracy: 0.7289
Epoch 6/20
200/200 [==============================] - 0s 749us/step - loss: 0.6181 - accuracy: 0.7333
Epoch 7/20
200/200 [==============================] - 0s 754us/step - loss: 0.6173 - accuracy: 0.7356
Epoch 8/20
200/200 [==============================] - 0s 841us/step - loss: 0.6170 - accuracy: 0.7351
Epoch 9/20
200/200 [==============================] - 0s 775us/step - loss: 0.6174 - accuracy: 0.7347
Epoch 10/20
200/200 [==============================] - 0s 754us/step - loss: 0.617

#### Train with important features from EDA

In [36]:
X =df[
    # vol_cols+prev_mid_m_cols+['bs_pressure1']
    ['ss_lob_1_v', 'bs_lob_1_v', 'l1_diff', 'momentum_up']
] 
Y = df['midprice_up']
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(len(X.columns),)),
    keras.layers.Dense(10, activation="elu"),
    keras.layers.Dense(10, activation="elu"),
    keras.layers.Dense(1, activation="sigmoid")
    ])
# model.summary()
eta = 0.025
model.compile(optimizer=keras.optimizers.Adam(learning_rate=eta), loss="binary_crossentropy", metrics=["accuracy"])
model.fit(X, Y, batch_size=500, epochs=20)

Epoch 1/20
200/200 [==============================] - 1s 925us/step - loss: 0.6420 - accuracy: 0.6403
Epoch 2/20
200/200 [==============================] - 0s 920us/step - loss: 0.5887 - accuracy: 0.7315
Epoch 3/20
200/200 [==============================] - 0s 900us/step - loss: 0.5780 - accuracy: 0.7328
Epoch 4/20
200/200 [==============================] - 0s 879us/step - loss: 0.5613 - accuracy: 0.7361
Epoch 5/20
200/200 [==============================] - 0s 945us/step - loss: 0.5374 - accuracy: 0.7413
Epoch 6/20
200/200 [==============================] - 0s 925us/step - loss: 0.5296 - accuracy: 0.7455
Epoch 7/20
200/200 [==============================] - 0s 884us/step - loss: 0.5294 - accuracy: 0.7429
Epoch 8/20
200/200 [==============================] - 0s 913us/step - loss: 0.5248 - accuracy: 0.7463
Epoch 9/20
200/200 [==============================] - 0s 885us/step - loss: 0.5231 - accuracy: 0.7478
Epoch 10/20
200/200 [==============================] - 0s 874us/step - loss: 0.523

In [33]:
X =df.drop(columns=['midprice_up'])

Y=df['midprice_up']

model = keras.Sequential([
    InputLayer(input_shape=(len(X.columns))),
    Dense(units=22, activation='elu'),
    Dense(units=22, activation='elu'),
    Dense(units=1, activation='sigmoid')
])

eta = 0.01
model.compile(optimizer=keras.optimizers.Adam(learning_rate=eta), loss="binary_crossentropy", metrics=["accuracy"])
model.fit(X, Y, batch_size=100, epochs=10)

Epoch 1/10
1000/1000 [==============================] - 1s 791us/step - loss: nan - accuracy: 0.5167
Epoch 2/10
1000/1000 [==============================] - 1s 785us/step - loss: nan - accuracy: 0.4973
Epoch 3/10
1000/1000 [==============================] - 1s 801us/step - loss: nan - accuracy: 0.4973
Epoch 4/10
1000/1000 [==============================] - 1s 785us/step - loss: nan - accuracy: 0.4973
Epoch 5/10
1000/1000 [==============================] - 1s 793us/step - loss: nan - accuracy: 0.4973
Epoch 6/10
1000/1000 [==============================] - 1s 802us/step - loss: nan - accuracy: 0.4973
Epoch 7/10
1000/1000 [==============================] - 1s 791us/step - loss: nan - accuracy: 0.4973
Epoch 8/10
1000/1000 [==============================] - 1s 809us/step - loss: nan - accuracy: 0.4973
Epoch 9/10
1000/1000 [==============================] - 1s 784us/step - loss: nan - accuracy: 0.4973
Epoch 10/10
1000/1000 [==============================] - 1s 788us/step - loss: nan - accura